In [1]:
import ee
import geemap
import logging
import multiprocessing
import os
import requests
import shutil
from retry import retry

In [2]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AbUR2VN6j3h_sKh8i2HQ8l-ic-xX_vUBVqmAWaS4Dj-97EEQiUjh8YRst7k

Successfully saved authorization token.


In [3]:
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

In [4]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [5]:
ds = ee.FeatureCollection("projects/ee-pnolaraj/assets/Ktm_Shp")
Map.addLayer(ds, {}, "Kathmandu")

In [6]:
# Provide any string on the name for filtering
def School_CoOrdinates(String):
    filtered_features = ds.filter(ee.Filter.stringContains('name', String))

    # Map over the filtered features to extract their coordinates
    coordinates_list = filtered_features.geometry().coordinates()
    
    #Places and Co ordinates values    
    place_names = filtered_features.aggregate_array('name').getInfo()
    CoOrdinatesList = coordinates_list.getInfo()
   
    
    #Sorting the boundary Cordinates to point
    CoOrdinates_sorted = []
    for i in CoOrdinatesList:
        for j in i:
            for k in j:
                CoOrdinates_sorted.append([k[0], k[1]])
                break
            break
            
            
    return [CoOrdinates_sorted, place_names]
    


In [7]:
String = "University"

Result = School_CoOrdinates(String)
CoOrdinate, Place_Name = Result[0], Result[1]

In [8]:
print(CoOrdinate)
print(Place_Name)

[[85.29523222592344, 27.67989212374201], [85.33953343309297, 27.708711248927987], [85.28451700918667, 27.684690056435628], [85.29727901658055, 27.67843399390603], [85.2815829473634, 27.682897547392695]]
['Tribhuwan University Office of Rector', 'Pokhara University (Kathmandu)', 'National Innovation Centre, Tribhuwan University', 'Nepal University Teachers Association', 'Tribhuvan University']


In [9]:
params = {
    'count': 100,  # How many image chips to export
    'buffer': 100,  # The buffer distance (m) around each point
    'scale': 100,  # The scale to do stratified sampling
    'seed': 1,  # A randomization seed to use for subsampling.
    'dimensions': '256x256',  # The dimension of each image chip
    'format': "geojson",  # The output image format, can be png, jpg, ZIPPED_GEO_TIFF, GEO_TIFF, NPY
    'prefix': f'{String}_',  # The filename prefix
    'processes': 25,  # How many processes to used for parallel processing
    'out_dir': 'Output',  # The output directory. Default to the current working directly
}

In [ ]:
image = ee.FeatureCollection("GOOGLE/Research/open-buildings/v2/polygons")

In [14]:
def GEOJSON_from_Point(Point, Name):
    image = ee.FeatureCollection("GOOGLE/Research/open-buildings/v2/polygons")
    point_geom = ee.Geometry.Point(Point[0], Point[1])
    buffer = point_geom.buffer(buffer_radius)
    clipped_collection = image.filterBounds(buffer)
    download_url = clipped_collection.getDownloadURL(filetype='geojson')
    
    r = requests.get(download_url, stream=True)
    if r.status_code != 200:
        r.raise_for_status()
    ext = params['format']
    out_dir = os.path.abspath(params['out_dir'])
    basename = str(Name)
    filename = f"{out_dir}/{params['prefix']}{basename}.{ext}"
    with open(filename, 'wb') as out_file:
        shutil.copyfileobj(r.raw, out_file)
    print("Done: ", basename)
    

In [ ]:
buffer_radius = 30
for Point, Name in zip(CoOrdinate, Place_Name):
    print(Point)
    GEOJSON_from_Point(Point, Name)

[85.29523222592344, 27.67989212374201]
Done:  Tribhuwan University Office of Rector
[85.33953343309297, 27.708711248927987]
Done:  Pokhara University (Kathmandu)
[85.28451700918667, 27.684690056435628]
